<a href="https://colab.research.google.com/github/amirpaia/blenderbot/blob/main/training_transformer_french_reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installating kaggle and download the dataset

please upload kaggle.json

In [ ]:
# download from kaggle
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! kaggle datasets download breandan/french-reddit-discussion
! unzip french-reddit-discussion.zip

 99% 421M/426M [00:18<00:00, 24.1MB/s]
100% 426M/426M [00:18<00:00, 24.4MB/s]
Archive:  french-reddit-discussion.zip
  inflating: final_SPF_2.xml         
  inflating: spf.tar.gz              


# Data preparation

In [ ]:
import time
import lxml.etree as ET
import pandas as pd
import numpy as np

file_path = 'final_SPF_2.xml'
start=time.time()
#Initializes the parser
parser = ET.XMLParser(recover=True)
#Parses the file
tree = ET.parse(file_path, parser=parser)
xroot = tree.getroot()

#One conversation -> one line in the data array
dfcols = ['link_id', 'subreddit_id', 'uid',"comment_id",'score', 'parent_id', 'create_utc', 'text']
data =np.array(
    (
        [
            [
                [
                    node.attrib.get('link_id'),
                    node.attrib.get('subreddit_id'), 
                    node.getchildren()[j].get('uid'), 
                    node.getchildren()[j].get('comment_id'), 
                    node.getchildren()[j].get('score'), 
                    node.getchildren()[j].get('parent_id'), 
                    node.getchildren()[j].get('create_utc'),
                    node.getchildren()[j].text
                ] 
                for j in range(len(node.getchildren()))
            ] 
            for node in xroot
        ]
     ), 
    dtype=object)

print('number of conversations: ', data.shape[0])

#one comments -> one line in the data array
data=np.array([liste for conversation in data for liste in conversation], dtype=object)
print('number of comments: ',data.shape[0])

df_xml = pd.DataFrame(data=data, columns=dfcols)
print('All done in :',time.time()-start,' seconds')

df_xml.head()

number of conversations:  556622
number of comments:  1583083
All done in : 23.115362644195557  seconds


,link_id,subreddit_id,uid,comment_id,score,parent_id,create_utc,text
0,8r1kz,2qhjz,1688932,c0a62uj,3,8r1kz,1244576002,Ironie : l'article disant qu'on est plus capab...
1,8r1kz,2qhjz,786883,c0a6lmb,1,c0a62uj,1244621120,"Moi-même, j'ai dû me forcer pour arriver jusqu..."
2,8sncs,2qhjz,390497,c0aawpk,1,8sncs,1245076061,Service qui sera rendu au contribuable pour la...
3,8sncs,2qhjz,32884,c0aaxba,3,c0aawpk,1245077396,Eeeeh oui ! 70 millions pour une loi qui aura ...
4,8v13c,2qhjz,796919,c0aj3ov,2,8v13c,1245830384,Est-ce qu'elle a vraiment commis des crimes qu...


In [ ]:
# pd.set_option('display.max_rows', 100)

# df_xml[df_xml['comment_id'] == "c4ojtph"].text.to_string(index=False)
# df_xml[df_xml['link_id'] == "tpa2b"]

text:hello	labels:how are you
text:good	labels:bye	episode_done:True



## previous data preparation

In [ ]:
train_dialogs_in_parlai_format = ""
valid_dialogs_in_parlai_format = ""
test_dialogs_in_parlai_format = ""
number_of_dialogs = len(df_xml.groupby('link_id'))

for index, (key, dialog) in enumerate(df_xml.groupby('link_id')):
#     print(key, dialog)
    sorted_dialog = dialog.values[dialog.values[:, 6].argsort()]
    list_of_turns = [turn[7].replace("\n\n"," ").replace("\n", "") for turn in sorted_dialog]
    parlai_format = transfer_dialog(list_of_turns)
#     print(parlai_format)
    if index < number_of_dialogs * 0.8:
      train_dialogs_in_parlai_format +=parlai_format
    elif index < number_of_dialogs * 0.9:
      valid_dialogs_in_parlai_format +=parlai_format
    else:
      test_dialogs_in_parlai_format +=parlai_format

    if index % 10000 == 0: print(index)
print(index)

# !rm -R fr_reddit_dataset
!mkdir fr_reddit_dataset
with open("fr_reddit_dataset/data_train.txt","w") as f:
    f.write(train_dialogs_in_parlai_format)

with open("fr_reddit_dataset/data_valid.txt","w") as f:
    f.write(valid_dialogs_in_parlai_format)

with open("fr_reddit_dataset/data_test.txt","w") as f:
    f.write(test_dialogs_in_parlai_format)
print('done!')

index: 0 number of dialogs: 0
index: 10000 number of dialogs: 1551
index: 20000 number of dialogs: 2765
index: 30000 number of dialogs: 3954
index: 40000 number of dialogs: 5205
index: 50000 number of dialogs: 6527
index: 60000 number of dialogs: 7850
index: 66538 number of dialogs: 8850
train set: 7080, validation set: 885,test set: 885
done!


## new data preparation

In [ ]:
all_dialogs_in_parlai_format = []

for index, (key, dialog) in enumerate(df_xml.groupby('link_id')):
    # if the dialog has two speakers
    if (len(set([a[2] for a in dialog.values])) == 2) & \
        (sum([1 for a in dialog.values if (len(a[7]) > 300)]) == 0):
        # print(key, dialog)
        sorted_dialog = dialog.values[dialog.values[:, 6].argsort()]
        list_of_turns = [turn[7].replace("\n\n"," ").replace("\n", "") for turn in sorted_dialog]

        parlai_format = transfer_dialog(list_of_turns)
        all_dialogs_in_parlai_format.append(parlai_format)
        # print(parlai_format)

    number_of_dialogs = len(all_dialogs_in_parlai_format)
    if index % 10000 == 0: print(f"index: {index}", f"number of dialogs: {number_of_dialogs}" )
print(f"index: {index}", f"number of dialogs: {number_of_dialogs}" )

df = pd.DataFrame (all_dialogs_in_parlai_format, columns = ['dialog'])

train, valid, test = np.split(df.sample(frac=1, random_state=42), 
                                 [int(.8*len(df)), int(.9*len(df))])

print(f"train set: {len(train)}, validation set: {len(valid)},test set: {len(test)}")

data_folder = "fr_reddit_dataset_small"
!rm -R $data_folder
!mkdir $data_folder
with open(f"{data_folder}/data_train.txt","w") as f:
    f.write(''.join(a[0] for a in train.values))

with open(f"{data_folder}/data_valid.txt","w") as f:
    f.write(''.join(a[0] for a in valid.values))

with open("fr_reddit_dataset_small/data_test.txt","w") as f:
    f.write(''.join(a[0] for a in test.values))
print('done!')

In [ ]:
train.values[0][0]

"text:Oh ouiii ! Je sais pas pourquoi, mais en voyant le bordel du débat à 11 je m'étais dit qu'il y avait un très fort potentiel de YTP.  Hâte de voir le résultat final, merci du partage.\tlabels:Je suis assez impressionné par la qualité de son travail. Je n'ai pas trouvé d'autres poopeurs qui soient aussi bon.\tepisode_done:True\n"

# Installatin

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed May 18 07:14:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# mydrive_path = '/content/drive/MyDrive/colabs/blender-models/'
# !pip uninstall -q parlai
!pip install -q parlai
!pip install transformers

# Reddit Ids > Dialog > ParlAI format > Train/Valid/Test

In [ ]:
def transfer_dialog(d):
    if len(d)%2 !=0: d = d[:-1]
    t = ""
    for i in range(0,len(d),2):
        u1 = d[i]
        u2 = d[i+1]

        if (i+2) != len(d):
            t += "text:"+u1+"\t"+"labels:"+u2+"\n"
        else:
            t += "text:"+u1+"\t"+"labels:"+u2+"\t"+"episode_done:True"+"\n"
    return t
    
t = ['hello','how are you','good','bye']
print(transfer_dialog(t))

In [ ]:
def convert_line_to_list_of_ids(line):
    return line.split("\t")[1].replace("[","").replace("]","").replace("\n","").replace(" ","").replace("'","").split(",")

In [ ]:
import re
def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)
    
print( remove_urls("this is a test https://sdfs.sdfsdf.com/sdfsdf/sdfsdf/sd/sdfsdfs?bob=%20tree&jef=man lets see this too https://sdfsdf.fdf.com/sdf/f end"))

this is a test  lets see this too  end


In [ ]:
data_path = '/content/drive/MyDrive/colabs/aliae-workspace/datasets/'
with open(f"{data_path}french_reddit_all_dialog_turns_ids.txt") as f:
    lines = f.readlines()

In [ ]:
pd.set_option('display.max_colwidth', None)
# print(len(lines))
# print(sum([1 for l in lines if len(convert_line_to_list_of_ids(l)) == 10]))
print(len(lines))

dict_xml = df_xml.set_index('comment_id').to_dict()['text']
dialogs_in_parlai_format = []
for index, line in enumerate(lines):
    ids = convert_line_to_list_of_ids(line)
    # print(ids)
    turns = []
    for id in ids:
        text = dict_xml[id].replace("\n", "").replace("\t", "")
        text = remove_urls(text)
        # print(text)
        turns.append(text)
    dialogs_in_parlai_format.append(transfer_dialog(turns))
    # print(f"index: {index}", f"number of turns: {len(turns)}")
    if index % 50000 == 0: print(f"index: {index}")

274561
index: 0
index: 50000
index: 100000
index: 150000
index: 200000
index: 250000


In [ ]:
data_path = '/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_reddit/'
# with open(data_path + "train.txt", "w") as f:
#     f.writelines(dialogs_in_parlai_format)

df = pd.DataFrame (dialogs_in_parlai_format, columns = ['dialog'])
train, valid, test = np.split(df.sample(frac=1, random_state=42), 
                                 [int(.8*len(df)), 
                                  int(.9*len(df))])
print(f"train set: {len(train)}, validation set: {len(valid)},test set: {len(test)}")

with open(f"{data_path}/data_train.txt","w") as f:
    f.write('\n'.join(a[0] for a in train.values))

with open(f"{data_path}/data_valid.txt","w") as f:
    f.write('\n'.join(a[0] for a in valid.values))

with open(f"{data_path}/data_test.txt","w") as f:
    f.write('\n'.join(a[0] for a in test.values))
print('done!')

train set: 219648, validation set: 27456,test set: 27457
done!


# Pre-training

In [ ]:
data_path = '/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_reddit/'
model_path = "/content/drive/MyDrive/colabs/blender-models/trained_fr_reddit/"
# !rm -R $model_path
# !mkdir $model_path

from parlai.scripts.train_model import TrainModel
TrainModel.main(
    task= 'fromfile:parlaiformat', 
    fromfile_datapath='data',
    fromfile_datatype_extension=True,

    model= "transformer/generator",
    model_file= f"{model_path}model",
    
    # depend on your gpu
    validation_every_n_epochs=0.25, # veps= 0.25, 
    num_epochs = 3,
    # log_every_n_secs= 60,
    verbose = True,
    attention_dropout= 0.0, 
    batchsize= 8, 
    fp16= True, fp16_impl= "mem_efficient",
    
    # arguments we get from the pretrained model. "from recipes page for 2.7B model" 
    embedding_size= 2560, ffn_size= 10240,
    variant= "prelayernorm",
    n_heads= 32, n_positions= 128, 
    n_encoder_layers= 2, n_decoder_layers= 24,
    
    label_truncate= 128, text_truncate= 128, truncate= 128,
    activation= "gelu",
    history_add_global_end_token= "end", 
    delimiter= '  ', 
    # dict_tokenizer= "bytelevelbpe",
    dropout= 0.1,

    # some training arguments, specific to this fine-tuning
    lr= 7e-06, lr_scheduler= "reduceonplateau", lr_scheduler_patience= 3,
    optimizer= "adam",
    relu_dropout= 0.0, 
    model_parallel= True,
    warmup_updates= 100,
    update_freq= 2,
    gradient_clip= 0.1, 
    # save_after_valid= True,

    # speeds up validation
    skip_generation= True,
    vp= 10,
)


## TrainModel

In [ ]:
model_path = "/content/drive/MyDrive/colabs/blender-models/trained_fr_reddit/"
data_path = '/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_reddit/'
# !rm -R $model_path
# !mkdir $model_path

from parlai.scripts.train_model import TrainModel
TrainModel.main(
    model_file= f"{model_path}model",
    task= 'fromfile:parlaiformat', 
    fromfile_datapath= f'{data_path}data',
    fromfile_datatype_extension=True,

    batchsize=12,
    
    model= "transformer/generator",

    n_heads=16, n_layers=8, n_positions=512, text_truncate=512,
    label_truncate=128, ffn_size=2048, embedding_size=512,
    activation='gelu', variant='xlm',
    dict_lower=True, dict_tokenizer='bpe',
    learn_positional_embeddings=True,

    # some training arguments, specific to this fine-tuning
    # use a small learning rate with ADAM optimizer
    lr=1e-5, optimizer='adam',
    warmup_updates=100,
    # early stopping on perplexity
    validation_metric='ppl',

    num_epochs = 5,
    validation_every_n_epochs=2,

    # depend on your gpu. If you have a V100, this is good
    fp16=True, fp16_impl='mem_efficient',

    # speeds up validation
    skip_generation=True,

    # helps us cram more examples into our gpu at a time
    dynamic_batching='full',
)

21:23:09 | building dictionary first...
21:23:09 | Overriding opt["optimizer"] to adam (previously: mem_eff_adam)
21:23:09 | Overriding opt["num_epochs"] to 5.0 (previously: 10.0)
21:23:09 | Using CUDA
21:23:09 | loading dictionary from /content/drive/MyDrive/colabs/blender-models/trained_fr_reddit/model.dict
21:23:09 | num words = 30051
21:23:10 | Total parameters: 74,766,336 (74,766,336 trainable)
21:23:10 | Loading existing model params from /content/drive/MyDrive/colabs/blender-models/trained_fr_reddit/model
21:23:13 | Opt:
21:23:13 |     activation: gelu
21:23:13 |     adafactor_eps: '[1e-30, 0.001]'
21:23:13 |     adam_eps: 1e-08
21:23:13 |     add_p1_after_newln: False
21:23:13 |     aggregate_micro: False
21:23:13 |     allow_missing_init_opts: False
21:23:13 |     attention_dropout: 0.0
21:23:13 |     batchsize: 12
21:23:13 |     beam_block_full_context: True
21:23:13 |     beam_block_list_filename: None
21:23:13 |     beam_block_ngram: -1
21:23:13 |     beam_context_block_ngr

({'clen': AverageMetric(237.1),
  'ctpb': GlobalAverageMetric(4259),
  'ctps': GlobalTimerMetric(4.445e+04),
  'ctrunc': AverageMetric(0.1119),
  'ctrunclen': AverageMetric(55.91),
  'exps': GlobalTimerMetric(245.3),
  'exs': SumMetric(4.753e+04),
  'gpu_mem': GlobalAverageMetric(0.1426),
  'llen': AverageMetric(75.36),
  'loss': AverageMetric(5.233),
  'lr': GlobalAverageMetric(9.9e-06),
  'ltpb': GlobalAverageMetric(1311),
  'ltps': GlobalTimerMetric(1.369e+04),
  'ltrunc': AverageMetric(0.1505),
  'ltrunclen': AverageMetric(19.59),
  'ppl': PPLMetric(187.4),
  'token_acc': AverageMetric(0.2331),
  'token_em': AverageMetric(0),
  'total_train_updates': GlobalFixedMetric(8.076e+04),
  'tpb': GlobalAverageMetric(5570),
  'tps': GlobalTimerMetric(5.814e+04)},
 {'clen': AverageMetric(241.6),
  'ctpb': GlobalAverageMetric(4265),
  'ctps': GlobalTimerMetric(4.49e+04),
  'ctrunc': AverageMetric(0.113),
  'ctrunclen': AverageMetric(59.74),
  'exps': GlobalTimerMetric(246.9),
  'exs': SumMetr

## DisplayModel

In [6]:
# model_path = "trained_fr_reddit-small/"
# data_path = "fr_reddit_dataset/"

model_path = "/content/drive/MyDrive/colabs/blender-models/trained_fr_reddit/"
data_path = '/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_reddit/'
print(f'{model_path}/model')
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='fromfile:parlaiformat', 
    fromfile_datapath= f"{data_path}data",
    fromfile_datatype_extension=True,

    # model= "transformer/generator",
    model_file= f"{model_path}model",
    # init_model= "zoo:blender/blender_400Mdistill/model",
    # dict_file= f"zoo:blender/blender_400Mdistill/model.dict",

    datatype= "test",
    # fromfile_datatype_extension= True,

    num_examples=20,
    skip_generation=False,

    # inference = 'topk', # Generation algorithm. Choices: beam, topk, greedy, delayedbeam, nucleus
    # temperature = 0.7, # Temperature to add during decoding. Default 1.0
    # topk=30, # K used in Top K sampling
    # beam_length_penalty=1.03 # Applies a length penalty. Set to 0 for no penalty. Default: 0.65.

    beam_context_block_ngram= 3,
    beam_block_ngram= 3, beam_min_length= 20, beam_size= 10,
    inference =  'topk', topk=20, temperature = 0.5, beam_length_penalty=0.8
)

/content/drive/MyDrive/colabs/blender-models/trained_fr_reddit//model
07:16:23 | Overriding opt["datatype"] to test (previously: train)
07:16:23 | Overriding opt["skip_generation"] to False (previously: True)
07:16:23 | Overriding opt["beam_context_block_ngram"] to 3 (previously: -1)
07:16:23 | Overriding opt["beam_block_ngram"] to 3 (previously: -1)
07:16:23 | Overriding opt["beam_min_length"] to 20 (previously: 1)
07:16:23 | Overriding opt["beam_size"] to 10 (previously: 1)
07:16:23 | Overriding opt["inference"] to topk (previously: greedy)
07:16:23 | Overriding opt["topk"] to 20 (previously: 10)
07:16:23 | Overriding opt["temperature"] to 0.5 (previously: 1.0)
07:16:23 | Overriding opt["beam_length_penalty"] to 0.8 (previously: 0.65)
07:16:23 | Using CUDA
07:16:23 | loading dictionary from /content/drive/MyDrive/colabs/blender-models/trained_fr_reddit/model.dict
07:16:24 | num words = 30051
07:16:25 | DEPRECATED: XLM should only be used for backwards compatibility, as it involves a 